# Instruction fine-tuning Llama 2 with PEFT’s QLoRa method

In [ ]:
# !pip install -q huggingface_hub
# !pip install -q -U trl transformers accelerate peft
# !pip install -q -U datasets bitsandbytes einops wandb
# !pip install ipywidgets

In [ ]:
# Uncomment to install new features that support latest models like Llama 2
# !pip install git+https://github.com/huggingface/peft.git
# !pip install git+https://github.com/huggingface/transformers.git

In [ ]:
# When prompted, paste the HF access token you created earlier.
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer

In [ ]:
dataset_name = "mychen76/small_openassistant-guanaco"
dataset = load_dataset(dataset_name, split="train")

base_model_name = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
device_map = {"": 0}

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=bnb_config,
    device_map=device_map,
    trust_remote_code=True,
    use_auth_token=True
)
base_model.config.use_cache = False

# More info: https://github.com/huggingface/transformers/pull/24906
base_model.config.pretraining_tp = 1 

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    max_steps=500
)

max_seq_length = 512

trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

trainer.train()

In [ ]:
import os
output_dir = os.path.join(output_dir, "final_checkpoint")
trainer.model.save_pretrained(output_dir)

# Quick test fine-tuned model
and dirty approach to load the model and do a sanity test.

In [ ]:
from datasets import load_dataset
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, TrainingArguments
from peft import LoraConfig
from trl import SFTTrainer
# Free memory for merging weights
#del model
#del trainer
torch.cuda.empty_cache()
output_dir='./results/final_checkpoint'
device_map = {"": 0}
from peft import AutoPeftModelForCausalLM

base_model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoPeftModelForCausalLM.from_pretrained(output_dir, device_map=device_map, torch_dtype=torch.bfloat16)

In [ ]:
text = "what color is the sky?"
inputs = tokenizer(text, return_tensors="pt").to(device_map)
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), attention_mask=inputs["attention_mask"], max_new_tokens=50, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Free memory for merging weights
torch.cuda.empty_cache()   
if model is not None:
    del model
if tokenizer is not None:
    del tokenizer 

## Load and Test Fined-Tuned Model

In [1]:
## properway to load fine-tuned models
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer

model_name = "meta-llama/Llama-2-7b-chat-hf"
adapters_name = "mychen76/Llama-2-7b-hf-guanaco-sm"

print(f"Starting to load the model {model_name} into memory")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #load_in_4bit=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
model = PeftModel.from_pretrained(model, adapters_name)
model = model.merge_and_unload()
tokenizer = LlamaTokenizer.from_pretrained(model_name)
tokenizer.bos_token_id = 1
stop_token_ids = [0]
print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model meta-llama/Llama-2-7b-chat-hf into memory


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Successfully loaded the model meta-llama/Llama-2-7b-chat-hf into memory


Then to use the model:
    

In [9]:
prompt = "Today was an amazing day because"
device_map={"": 0}
inputs = tokenizer(prompt, return_tensors="pt").to(device_map)
tokenizer.pad_token = tokenizer.eos_token
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), do_sample=True, num_beams=1, max_new_tokens=100)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

Attempting to cast a BatchEncoding to type {'': 0}. This is not supported.


["Today was an amazing day because I got to spend time with my favorite person in the whole world! 😍 We did so many fun things together, like going to the park, playing on the swings, and feeding the ducks. We even had a picnic lunch and took a nap together. 😴 It was just the best day ever! 😘 I'm so lucky to have such a wonderful person in my life. 💕 #"]

To use it for zero-shot classification:

In [11]:
# To use it for zero-shot classification:
from transformers import pipeline
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
classifier = pipeline("zero-shot-classification", model=model, tokenizer=tokenizer)
classifier("Today was an amazing day", candidate_labels=["negative", "positive"])

Failed to determine 'entailment' label id from the label2id mapping in the model config. Setting to -1. Define a descriptive label2id mapping in the model config to ensure correct outputs.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'sequence': 'Today was an amazing day',
 'labels': ['positive', 'negative'],
 'scores': [0.8899121284484863, 0.11008787900209427]}